# Runtime experiments and comparisions

In [ ]:

import itertools
import random
import timeit

import networkx as nx
import igraph as ig

import datasets.cosnology as cosnology
import datasets.email_enron as email_enron
import datasets.facebook_combined as facebook_combined
import datasets.brightkite_edges as brightkite
import datasets.roadnet_CA as roadnet_CA
import datasets.wiki_vote as wiki_vote


random.seed(42)

In [2]:
# graphs = [ nx.karate_club_graph(), jazz.get_graph(), cora.get_graph(), grcq.get_graph() ]
# graph_names = [ 'Karate Club', 'Jazz Musicians', 'Cora Citations', 'Arxiv GR-QC' ]

graphs = [ nx.karate_club_graph(), cosnology.get_graph(), email_enron.get_graph(), facebook_combined.get_graph(), brightkite.get_graph(), roadnet_CA.get_graph(), wiki_vote.get_graph() ]
graphs = [ig.Graph.from_networkx(g) for g in graphs]
graph_names = [ 'Karate Club', 'Cosnology', 'Email Enron', 'Facebook', 'Brightkite', 'Roadnet CA', 'Wiki Vote' ]

In [3]:
for graph, graph_name in zip(graphs, graph_names):
    print(f'{graph_name}: {str(graph.summary())} ')

Karate Club: IGRAPH U-W- 34 78 -- Zachary's Karate Club
+ attr: name (g), _nx_name (v), club (v), weight (e) 
Cosnology: IGRAPH U--- 5242 14496 -- 
+ attr: _nx_name (v) 
Email Enron: IGRAPH U--- 36692 183831 -- 
+ attr: _nx_name (v) 
Facebook: IGRAPH U--- 4039 88234 -- 
+ attr: _nx_name (v) 
Brightkite: IGRAPH U--- 58228 214078 -- 
+ attr: _nx_name (v) 
Roadnet CA: IGRAPH U--- 1965206 2766607 -- 
+ attr: _nx_name (v) 
Wiki Vote: IGRAPH U--- 7115 100762 -- 
+ attr: _nx_name (v) 


In [4]:
import leidenalg
import louvain
# 𝓗 = Modularity(1.0)

fn_louvain_mod = lambda G: louvain.find_partition(G, louvain.ModularityVertexPartition)
fn_leiden_mod  = lambda G: leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)

# 𝓗 = CPM(0.95)
fn_louvain_cpm = lambda G: louvain.find_partition(G, louvain.CPMVertexPartition)
fn_leiden_cpm  = lambda G: leidenalg.find_partition(G, leidenalg.CPMVertexPartition)


algorithms = [ fn_louvain_mod, fn_louvain_cpm, fn_leiden_mod, fn_leiden_cpm ]
algo_names = [ 'Louvain (Mod)', 'Louvain (CPM)', 'Leiden (Mod)', 'Leiden (CPM)']

/var/folders/xc/yp5mkx455gq2n2b658qm_3_c0000gn/T/ipykernel_2642/1365607280.py:2: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


In [5]:
times = {}

In [6]:


for ((graph, g_name), (algo, a_name)) in itertools.product(zip(graphs, graph_names), zip(algorithms, algo_names)):
    print(f"Running {a_name} on {g_name} … ", end='')
    # First, build a callable, that will repeatedly be run to measure the average execution time:
    test_callable = lambda: algo(graph)
    time = min(timeit.repeat(stmt=test_callable, repeat=5, number=1))
    times[(a_name,g_name)] = time
    print(f"execution time: ~ {time:.6f} seconds.")

Running Louvain (Mod) on Karate Club … execution time: ~ 0.000269 seconds.
Running Louvain (CPM) on Karate Club … execution time: ~ 0.000087 seconds.
Running Leiden (Mod) on Karate Club … execution time: ~ 0.000869 seconds.
Running Leiden (CPM) on Karate Club … execution time: ~ 0.000416 seconds.
Running Louvain (Mod) on Cosnology … execution time: ~ 0.043044 seconds.
Running Louvain (CPM) on Cosnology … execution time: ~ 0.011175 seconds.
Running Leiden (Mod) on Cosnology … execution time: ~ 0.100865 seconds.
Running Leiden (CPM) on Cosnology … execution time: ~ 0.052582 seconds.
Running Louvain (Mod) on Email Enron … execution time: ~ 0.603307 seconds.
Running Louvain (CPM) on Email Enron … execution time: ~ 0.116740 seconds.
Running Leiden (Mod) on Email Enron … execution time: ~ 1.139032 seconds.
Running Leiden (CPM) on Email Enron … execution time: ~ 0.510745 seconds.
Running Louvain (Mod) on Facebook … execution time: ~ 0.075346 seconds.
Running Louvain (CPM) on Facebook … execut

In [ ]:
print("Graph | Size | Louvain (Mod) | Louvain (CPM) | Leiden (Mod) | Leiden (CPM)")

for graph, graph_name in zip(graphs, graph_names):
    print(f"{graph_name: >14}", end='')
    print(f" | {str(graph.vcount()) + ' / ' + str(graph.ecount()): ^14}", end='')
    for algo in algo_names:
        time = times[(algo, graph_name)]
        print(f" | {time: >12.6f} s", end='')
    print()
    


Graph | Size | Louvain (Mod) | Louvain (CPM) | Leiden (Mod) | Leiden (CPM)
   Karate Club |    34 / 78     |     0.000269 s |     0.000087 s |     0.000869 s |     0.000416 s
     Cosnology |  5242 / 14496  |     0.043044 s |     0.011175 s |     0.100865 s |     0.052582 s
   Email Enron | 36692 / 183831 |     0.603307 s |     0.116740 s |     1.139032 s |     0.510745 s
      Facebook |  4039 / 88234  |     0.075346 s |     0.034451 s |     0.208009 s |     0.147571 s
    Brightkite | 58228 / 214078 |     0.870898 s |     0.158342 s |     1.685763 s |     0.713833 s
    Roadnet CA | 1965206 / 2766607 |    45.983364 s |     4.991504 s |    47.618056 s |    21.674825 s
     Wiki Vote | 7115 / 100762  |     0.089668 s |     0.044532 s |     0.388204 s |     0.188090 s


These tests were run on Macbook Air (2020) M1 with 16GB RAM, macOS Sequoia 15.1.1 (Darwin Kernel Version 24.1.0)